In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('Data.csv')

In [5]:
data

,County,HospitalID,MedicareFr,RPR,RER
0,Alameda,50002,0.385420,24.126127,22.505701
1,Humboldt,50006,0.463131,16.193045,19.058270
2,San Mateo,50007,0.302269,20.862410,20.750566
3,Napa,50009,0.430666,20.507330,22.007002
4,Napa,50013,0.487894,20.139364,21.881418
...,...,...,...,...,...
178,Orange,50746,0.150837,20.370153,21.221684
179,Los Angeles,50752,0.544773,25.725578,24.427887
180,San Diego,50757,0.458330,20.635431,21.956419
181,Los Angeles,50761,0.508488,22.451075,22.105062


In [8]:
# RPR: actual readmissions performace measure
# RER: target performance measure
# ERR: Excess Readmission Ratio = RPR/RER

# if ERR>1: financial penalty = (ERR-1)*CMS*k, k is the proportion
# Here we set k = 1
# penalty max proportion = 1-FAF(97%) = 0.03

# CMS: Medicare & Medicaid Services payment
# MedicareFr: Fraction of medicare patients (others are private patients)

In [6]:
# create two empty lists to store the maximum revenue and current revenue
revenue_max = list()
revenue_now = list()

for i in range(len(data)):
    # Find the RPR and RER values of each row
    rpr = data.loc[i, 'RPR']
    rer = data.loc[i, 'RER']
    
    # The fraction of medicare patients
    medifrac = data.loc[i, 'MedicareFr']
    
    # Calculate the factor proportion of panelty(What about CMS?)
    # "it incurs a financial penalty proportional to the level of excessiveness"
    # Set the proportion of excessiveness(k) to 1
    # If excessiveness >1, panelty proportion = rpr/rer-1
    
    # max(k(rpr/rer -1), 0)
    penalty = min(max(rpr/rer - 1, 0), 0.03)
    
    # !!Create a model of the revenue(??!!!)
    revenue_current=(1 - penalty * medifrac)*(1 + rpr/100)
    revenue_now.append(revenue_current)
    
    # Now find the max revenue of the hospital
    # Find this point by create 50000 points from the interval[1, rpr)
    rpr_test = np.linspace(1, rpr, 50000, endpoint = False)
    revenue_list = list()
    
    for j in rpr_test:
        penalty2 = min(max(j/rer - 1, 0), 0.03)
        
        #What is the model????
        revenue2 = (1 - penalty2 * medifrac)*(1 + j/100)
        revenue_list.append(revenue2)       
    # Find the maximun revenue in revenue_list
    revenue_max.append(max(revenue_list))

Set patient volume = V
    Medicare patients = medifrac*V
    privtate patients = (1 - medifrac)*V

(medifrac*V*p(1-penalty)+(1 - medifrac)*p*V)*(1+rpr/100)
(medifrac(1-penalty)+(1 - medifrac))*(1+rpr/100)*Vp
->
(1 - medifrac * panelty)*(1+rpr/100)*Vp

In [7]:
revenue_max[:5]

[1.2269045010347992,
 1.161927407991068,
 1.2075040226791405,
 1.205069397934012,
 1.201389814527152]

In [8]:
revenue_now[:5]

[1.226909072780645,
 1.1619304466,
 1.2066550154188262,
 1.2050732994,
 1.2013936424]

In [9]:
dic = {'max': revenue_max, 'now': revenue_now}
df = pd.DataFrame(dic)

In [10]:
df

,max,now
0,1.226905,1.226909
1,1.161927,1.161930
2,1.207504,1.206655
3,1.205069,1.205073
4,1.201390,1.201394
...,...,...
178,1.203698,1.203702
179,1.244277,1.236708
180,1.206350,1.206354
181,1.221049,1.214764


In [11]:
df['incentive'] = df['max'] > df['now']

In [12]:
df

,max,now,incentive
0,1.226905,1.226909,False
1,1.161927,1.161930,False
2,1.207504,1.206655,True
3,1.205069,1.205073,False
4,1.201390,1.201394,False
...,...,...,...
178,1.203698,1.203702,False
179,1.244277,1.236708,True
180,1.206350,1.206354,False
181,1.221049,1.214764,True


In [13]:
# Count how many hospitals' maximum revenue is larger than current revenue
sum(df['incentive'] == True)

65

In [15]:
df.to_csv('compare_table.csv')